<a href="https://colab.research.google.com/github/Lyynn777/VibeHue/blob/main/VibeHueNB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch pillow matplotlib --quiet


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np

# Model A (7 emotions)
tokA = AutoTokenizer.from_pretrained("j-hartmann/emotion-english-distilroberta-base")
modelA = AutoModelForSequenceClassification.from_pretrained("j-hartmann/emotion-english-distilroberta-base")

# Model B (public, 6-emotion model)
tokB = AutoTokenizer.from_pretrained("bhadresh-savani/distilbert-base-uncased-emotion")
modelB = AutoModelForSequenceClassification.from_pretrained("bhadresh-savani/distilbert-base-uncased-emotion")


In [ ]:
# Color mapping
emotion_color_map = {
    "anger":      (220, 30, 30),
    "fear":       (150, 80, 180),
    "joy":        (255, 190, 60),
    "love":       (255, 150, 180),
    "sadness":    (60, 100, 160),
    "surprise":   (255, 120, 80),
    "neutral":    (150, 150, 150),
}

def predict_A(text):
    inp = tokA(text, return_tensors="pt")
    with torch.no_grad():
        logits = modelA(**inp).logits
    probs = torch.softmax(logits, dim=1)[0].tolist()
    labels = list(modelA.config.id2label.values())
    idx = int(np.argmax(probs))
    return labels[idx], probs[idx]

def predict_B(text):
    inp = tokB(text, return_tensors="pt")
    with torch.no_grad():
        logits = modelB(**inp).logits
    probs = torch.softmax(logits, dim=1)[0].tolist()
    labels = list(modelB.config.id2label.values())
    idx = int(np.argmax(probs))
    return labels[idx], probs[idx]


# Map Model B → Model A’s emotion family
B_to_core = {
    "anger": "anger",
    "fear": "fear",
    "joy": "joy",
    "love": "love",
    "sadness": "sadness",
    "surprise": "surprise",
}

def unify_B(label):
    return B_to_core.get(label, "neutral")

def fused_emotion(text):
    eA, sA = predict_A(text)
    eB, sB = predict_B(text)

    coreA = eA
    coreB = unify_B(eB)

    scores = {}
    scores[coreA] = scores.get(coreA, 0) + sA * 0.55
    scores[coreB] = scores.get(coreB, 0) + sB * 0.45

    final = max(scores, key=scores.get)
    confidence = scores[final]

    return final, confidence, (eA, sA, eB, sB)


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

def make_palette(color):
    r, g, b = color
    base = np.array([r, g, b])
    shifts = [30, -20, 50, -60]
    colors = [base]
    for s in shifts:
        colors.append((base + s) % 255)
    return [tuple(map(int, c)) for c in colors]

def draw_palette(colors):
    w, h = 900, 200
    stripe = w // len(colors)
    img = Image.new("RGB", (w, h))
    for i, c in enumerate(colors):
        part = Image.new("RGB", (stripe, h), c)
        img.paste(part, (i * stripe, 0))
    return img


In [ ]:
text = input("Describe your mood: ")

final_emotion, confidence, raw = fused_emotion(text)

# Get color
color = emotion_color_map.get(final_emotion, (150,150,150))
palette = make_palette(color)
img = draw_palette(palette)

print("\n=== Emotion Ensemble Result ===")
print(f"Model A → {raw[0]} ({raw[1]*100:.1f}%)")
print(f"Model B → {raw[2]} ({raw[3]*100:.1f}%)")

print(f"\nFinal Emotion: {final_emotion.upper()}")
print(f"Confidence: {confidence*100:.1f}%\n")

plt.figure(figsize=(10,2))
plt.imshow(img)
plt.axis("off")
plt.show()

img.save("mood_palette.png")
print("Saved as mood_palette.png")
